# Reinforecement Learning for Traffic Light Control

Required packages or environments:

* Python 3.6
* SUMO 0.32 with TraCI module. Click the [link](https://sumo.dlr.de/docs/SUMO_User_Documentation.html) for user documentation 
* Keras 2.2.0 and Tensorflow 1.9.0

## Setting Up the environment through SUMO and TRACI

In [3]:
from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import optparse
import argparse
import random
from pprint import pprint as pp
import numpy as np

### Import python modules from the SUMO_HOME/tools directory

In [4]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

### Simulation environment setting up

In [5]:
from sumolib import checkBinary  
import traci  # noqa
import traci.constants as tc

### Generating route file (data/road.rou.xml) based on the demand per second from differnt directions

In [6]:
def generate_routefile():
    random.seed(42)  # make tests reproducible
    N = 3600  # number of time steps
    # demand per second from different directions
    pEB = 1. / 10
    pWB = 1. / 11
    pSB = 1. / 15
    pNB = 1. / 25
    with open("data/road.rou.xml", "w") as routes:
        print("""<routes>
        <vType id="typeWE" accel="0.8" decel="4.5" sigma="0.5" length="5" minGap="2.5" maxSpeed="16.67" \
guiShape="passenger"/>
        <vType id="typeNS" accel="0.8" decel="4.5" sigma="0.5" length="7" minGap="3" maxSpeed="25" guiShape="bus"/>

        <route id="EastBound" edges="W-0W 0W-0 0-0E 0E-E" />
        <route id="WestBound" edges="E-0E 0E-0 0-0W 0W-W" />
        <route id="SouthBound" edges="N-0N 0N-0 0-0S 0S-S" />
        <route id="NorthBound" edges="S-0S 0S-0 0-0N 0N-N" />""", file=routes)
        vehNr = 0
        for i in range(N):
            if random.uniform(0, 1) < pEB:
                print('    <vehicle id="EastBound_%i" type="typeWE" route="EastBound" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pWB:
                print('    <vehicle id="WestBound_%i" type="typeWE" route="WestBound" depart="%i" />' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pSB:
                print('    <vehicle id="SouthBound_%i" type="typeNS" route="SouthBound" depart="%i" color="1,0,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
            if random.uniform(0, 1) < pNB:
                print('    <vehicle id="NorthBound_%i" type="typeNS" route="NorthBound" depart="%i" color="1,0,0"/>' % (
                    vehNr, i), file=routes)
                vehNr += 1
        print("</routes>", file=routes)

In [7]:
# The program looks like this
#    <tlLogic id="tl0" type="static" programID="0" offset="0">
# the locations of the tls are      NESW
#        <phase duration="41" state="GGrrGGrr"/>
#        <phase duration="4" state="yyrryyrr"/>
#        <phase duration="41" state="rrGGrrGG"/>
#        <phase duration="4" state="rryyrryy"/>
#    </tlLogic>

In [8]:
def run():
    """execute the TraCI control loop"""
#    step = 0
    # we start with phase 2 where EW has green
   
    traci.junction.subscribeContext("tl0", tc.CMD_GET_VEHICLE_VARIABLE, 300, [tc.VAR_SPEED, tc.VAR_WAITING_TIME]) 
    print(traci.junction.getContextSubscriptionResults("tl0"))
#   traci.trafficlight.setPhase("tl0", 2)
#    while traci.lane.getLastStepVehicleNumber("0W-0_0") <200:
#   while traci.simulation.getMinExpectedNumber() > 0:        
    for step in range(500):
        print(f'\n_______________' \
              f"(step {step}):", end=' ')
        print(f"Delta = {traci.simulation.getDeltaT()} s", end='  _  ')
        print(f'# MinExpectedNumber: {traci.simulation.getMinExpectedNumber()}', end='  ')
        
        print(f'TL Phase {traci.trafficlight.getPhase("tl0")} _ ' \
              f'{traci.trafficlight.getRedYellowGreenState("tl0")}' \
              f'({traci.trafficlight.getPhaseDuration("tl0")} s) _ '\
              f'next phse at {traci.trafficlight.getNextSwitch("tl0")}s')
        
        traci.simulationStep()
        print(f' #EB approaching TL : {traci.edge.getLastStepVehicleNumber("0W-0")}' \
              f'  _  # vehicles waiting: {traci.edge.getLastStepHaltingNumber("0W-0")}' \
              f'EB ttl waiting time: {traci.edge.getWaitingTime("0W-0")}')
        print(traci.junction.getContextSubscriptionResults("tl0"))
#    if traci.trafficlight.getPhase("tl0") == 2:
            # we are not already switching
#           if traci.inductionloop.getLastStepVehicleNumber("tl0") > 0:
                # there is a vehicle from the north, switch
#       traci.trafficlight.setPhase("tl0", 3)
#           else:
                # otherwise try to keep green for EW
#               traci.trafficlight.setPhase("tl0", 2)
#        eb = traci.lane.getLastStepVehicleNumber("0W-0_0")
#        step += 1
#        print("Step "+ str(step))
#        print('Eastbound vechicles : ', eb)
    #traci.close()
    #sys.stdout.flush()


### TrafficSignal Class for traffic lights of an intersection,  via TRACI API to retriev status info and change traffice lights phases. 

In [9]:
class TrafficSignal:
    """
    This class represents a Traffic Lights of an intersection,
    via TRACI API to retriev status info and change traffice lights phases.
    """
    def __init__(self, tlsID,programID,delta_time, min_green, max_green, phases):
        self.id = tlsID
        self.programID = programID
        self.time_on_phase = 0.0
        self.delta_time = delta_time
        self.min_green = min_green
        self.max_green = max_green
        self.green_phase = 0
        self.num_green_phases = len(phases) // 2
        self.lanes = list(dict.fromkeys(traci.trafficlight.getControlledLanes(self.id)))  # remove duplicates and keep order
        self.edges = self._compute_edges()
        self.edges_capacity = self._compute_edges_capacity()

        logic = traci.trafficlight.Logic("new-program", 0, 0, phases=phases)
        traci.trafficlight.setCompleteRedYellowGreenDefinition(self.id, logic)


In [10]:
import argparse
def get_options():
    optParser = argparse.ArgumentParser()
    optParser.add_argument("--nogui", action="store_true",
                         default=False, help="run the commandline version of sumo")
#    options, args = optParser.parse_args()
    options, args = optParser.parse_known_args()
#    optParser.print_help()
    return options

this is the main entry point of the simulation

In [11]:
if __name__ == "__main__":
    options = get_options()
#    print(options)
    # this script has been called from the command line. It will start sumo as a
    # server, then connect and run
    if options.nogui:
        sumoBinary = checkBinary('sumo')
    else:
        sumoBinary = checkBinary('sumo-gui')

    # first, generate the route file for this simulation
    generate_routefile()

    # this is the normal way of using traci. sumo is started as a
    # subprocess and then the python script connects and runs
    traci.start([sumoBinary, "-c", "data/road.sumocfg",
                             "--tripinfo-output", "tripinfo.xml"])
    run()

 Retrying in 1 seconds
None

_______________(step 0): Delta = 1.0 s  _  # MinExpectedNumber: 2  TL Phase 0 _ GGrrGGrr(41.0 s) _ next phse at 41.0s
 #EB approaching TL : 0  _  # vehicles waiting: 0EB ttl waiting time: 0.0
None

_______________(step 1): Delta = 1.0 s  _  # MinExpectedNumber: 2  TL Phase 0 _ GGrrGGrr(41.0 s) _ next phse at 41.0s
 #EB approaching TL : 0  _  # vehicles waiting: 0EB ttl waiting time: 0.0
None

_______________(step 2): Delta = 1.0 s  _  # MinExpectedNumber: 2  TL Phase 0 _ GGrrGGrr(41.0 s) _ next phse at 41.0s
 #EB approaching TL : 0  _  # vehicles waiting: 0EB ttl waiting time: 0.0
None

_______________(step 3): Delta = 1.0 s  _  # MinExpectedNumber: 54  TL Phase 0 _ GGrrGGrr(41.0 s) _ next phse at 41.0s
 #EB approaching TL : 0  _  # vehicles waiting: 0EB ttl waiting time: 0.0
None

_______________(step 4): Delta = 1.0 s  _  # MinExpectedNumber: 54  TL Phase 0 _ GGrrGGrr(41.0 s) _ next phse at 41.0s
 #EB approaching TL : 0  _  # vehicles waiting: 0EB ttl wai

In [12]:
junc_id = traci.junction.getIDList()

In [18]:
class Env_TLC:
    """
    Class for Traffic Light Control Environment which is used for retrieving State,
    changing the traffic phases or duration via TRACI API

    """

    def __init__(self, programID, tlsID):
        self.ID = tlsID
        self.programID = programID
        self.lanes = traci.trafficlight.getControlledLanes(self.ID)
        self.links = traci.trafficlight.getControlledLinks(lgt_1)
        self.RYG_definition = traci.trafficlight.getCompleteRedYellowGreenDefinition(lgt_1)
        self.last_state = {}
        
        self.IBlaneList = [self.links[i][0][0]for i in range(len(self.links))]
        self.OBlaneList = [self.links[i][0][1]for i in range(len(self.links))]
        self.linkList = [self.links[i][0][2]for i in range(len(self.links))]
    
    def getStateArray(state):
        return np.row_stack([np.array(each) for each in tl0.last_state.values()])

    def updateLastState(self, ):

        self.last_state['IBOccupancy'] = [traci.lane.getLastStepOccupancy(
            laneID) for laneID in self.IBlaneList]
        self.last_state['IBVolume'] = [traci.lane.getLastStepVehicleNumber(
            laneID) for laneID in self.IBlaneList]
        self.last_state['IBMeanSpeed'] = [traci.lane.getLastStepMeanSpeed(
            laneID) for laneID in self.IBlaneList]
        self.last_state['IBQueuSize'] = [traci.lane.getLastStepHaltingNumber(
            laneID) for laneID in self.IBlaneList]
        self.last_state['IBWaitingTime'] = [traci.lane.getWaitingTime(
            laneID) for laneID in self.IBlaneList]

        self.last_state['OBOccupancy'] = [traci.lane.getLastStepOccupancy(
            laneID) for laneID in self.OBlaneList]
        self.last_state['OBVolume'] = [traci.lane.getLastStepVehicleNumber(
            laneID) for laneID in self.OBlaneList]
        self.last_state['OBMeanSpeed'] = [traci.lane.getLastStepMeanSpeed(
            laneID) for laneID in self.IBlaneList]
        self.last_state['OBQueuSize'] = [traci.lane.getLastStepHaltingNumber(
            laneID) for laneID in self.OBlaneList]
        self.last_state['OBWaitingTime'] = [traci.lane.getWaitingTime(
            laneID) for laneID in self.OBlaneList]

In [19]:
tfcLgts =traci.trafficlight.getIDList()
lgt_1 = tfcLgts[0]
light1_Logic = traci.trafficlight.getCompleteRedYellowGreenDefinition(lgt_1)[0]

In [21]:
tl0 = Env_TLC(programID=light1_Logic.programID, tlsID=lgt_1)

In [46]:
tl0.updateLastState()
state = tl0.getStateArray()

In [48]:
tl0.last_state.keys()

dict_keys(['IBOccupancy', 'IBVolume', 'IBMeanSpeed', 'IBQueuSize', 'IBWaitingTime', 'OBOccupancy', 'OBVolume', 'OBMeanSpeed', 'OBQueuSize', 'OBWaitingTime'])

In [47]:
state

array([[ 0.06898419,  0.        ,  0.05179558,  0.01726519,  0.        ,
         0.        ,  0.06906077,  0.        ],
       [ 3.        ,  0.        ,  3.        ,  1.        ,  0.        ,
         0.        ,  4.        ,  0.        ],
       [ 7.91120776, 16.67      ,  7.58671141,  3.99708962, 16.67      ,
        16.67      ,  7.16412045, 16.67      ],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        , 11.        ,  0.        ,  0.        ,
         0.        ,  5.        ,  0.        ],
       [ 0.02417127,  0.02417127,  0.01726519,  0.        ,  0.02417127,
         0.02417127,  0.        ,  0.        ],
       [ 1.        ,  1.        ,  1.        ,  0.        ,  1.        ,
         1.        ,  0.        ,  0.        ],
       [ 7.91120776, 16.67      ,  7.58671141,  3.99708962, 16.67      ,
        16.67      ,  7.16412045, 16.67      ],
       [ 0.        ,  0.        

In [19]:
observation = tl0.traffic_volume()

0
0.06898418718431845
3
0.0


In [17]:
[len(each) for each in observation]

[8, 8, 8, 8]

In [18]:
tl0.links[0][0]

('0N-0_0', '0-0S_0', ':tl0_0_0')

In [ ]:
np.zeros()